In [ ]:
# default_exp core

# Core

> Conatains methods for transferring.

In [ ]:
#hide
from nbdev.showdoc import *
from fastai.tabular.all import *

In [ ]:
# This will contain the functions for transferring embeddings

In [ ]:
path = untar_data(URLs.ADULT_SAMPLE)
df1 = pd.read_csv(path/'adult.csv')
splits1 = RandomSplitter(valid_pct=0.2)(range_of(df1))
to1 = TabularPandas(df1, procs=[Categorify, FillMissing,Normalize],
                   cat_names = ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race'],
                   cont_names = ['age', 'fnlwgt', 'education-num'],
                   y_names='salary',
                   splits=splits1)
dls1 = to1.dataloaders(bs=64)
learn1 = tabular_learner(dls1, metrics=accuracy)

In [ ]:
learn1.model.embeds

ModuleList(
  (0): Embedding(10, 6)
  (1): Embedding(17, 8)
  (2): Embedding(8, 5)
  (3): Embedding(16, 8)
  (4): Embedding(7, 5)
  (5): Embedding(6, 4)
  (6): Embedding(3, 3)
)

Aim is to take these embeddings and transfer them to other models

In [ ]:
learn1.model.embeds[0].weight, learn1.model.embeds[0].weight.shape

(Parameter containing:
 tensor([[-9.4948e-03, -8.0228e-03, -1.0030e-03,  4.9856e-03,  5.4568e-03,
           1.7855e-03],
         [-1.6155e-02, -1.9413e-03, -3.9701e-03, -1.2165e-02, -1.2602e-03,
          -8.0813e-03],
         [ 3.5964e-03,  5.6924e-03,  3.4328e-04, -1.1778e-02, -7.4721e-03,
           1.5772e-03],
         [-1.0661e-02, -2.9555e-03, -3.1351e-03,  2.2408e-03,  1.4735e-02,
          -3.2288e-03],
         [ 3.5296e-03, -6.1447e-03,  1.0373e-03, -1.9636e-02,  1.2360e-02,
           1.4212e-02],
         [-1.0681e-02,  1.3563e-03, -1.7049e-02,  2.7756e-03, -5.3467e-03,
           3.4818e-04],
         [ 6.8925e-03, -8.6594e-03,  1.2888e-03,  5.7663e-03,  9.4138e-04,
           2.8628e-03],
         [-1.1746e-02, -1.2486e-02, -1.1488e-04, -6.7977e-04, -8.0588e-03,
          -1.2513e-02],
         [ 5.6534e-05,  2.1546e-04, -7.2593e-03,  2.8732e-03, -1.3235e-03,
           7.4169e-03],
         [ 2.8654e-03,  3.2996e-03, -1.4163e-03,  3.0365e-03,  3.4331e-03,
          -

In [ ]:
#We add robot to our "race" column
new_rows = pd.DataFrame([[49,'Private',101320,'Assoc-acdm',12.0,'Married-civ-spouse','Exec-managerial','Wife','Robot','Female',0,1902,40,'United-States','>=50k'],
                        [18,'Private',182308,'Bachelors',10.0,'Never-married','?','Own-child','Other','Male',0,0,23,'United-States','<50k']],
                        columns=df1.columns)
df2 = df1.copy()
df2 = df2.append(new_rows, ignore_index=True)
df2.tail()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,salary
32558,53,Private,157069,Assoc-acdm,12.0,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0,0,40,United-States,>=50k
32559,32,Local-gov,217296,HS-grad,9.0,Married-civ-spouse,Transport-moving,Wife,White,Female,4064,0,22,United-States,<50k
32560,26,Private,182308,Some-college,10.0,Married-civ-spouse,Prof-specialty,Husband,White,Male,0,0,40,United-States,<50k
32561,49,Private,101320,Assoc-acdm,12.0,Married-civ-spouse,Exec-managerial,Wife,Robot,Female,0,1902,40,United-States,>=50k
32562,18,Private,182308,Bachelors,10.0,Never-married,?,Own-child,Other,Male,0,0,23,United-States,<50k


In [ ]:
splits2 = RandomSplitter(valid_pct=0.2)(range_of(df2))
to2 = TabularPandas(df2, procs=[Categorify, FillMissing,Normalize],
                   cat_names = ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race'],
                   cont_names = ['age', 'fnlwgt', 'education-num'],
                   y_names='salary',
                   splits=splits2)
dls2 = to2.dataloaders(bs=64)
learn2 = tabular_learner(dls2, metrics=accuracy)

In [ ]:
learn2.model.embeds

ModuleList(
  (0): Embedding(11, 6)
  (1): Embedding(19, 8)
  (2): Embedding(10, 6)
  (3): Embedding(18, 8)
  (4): Embedding(9, 5)
  (5): Embedding(8, 5)
  (6): Embedding(3, 3)
)

In [ ]:
learn1.dls.cat_names

(#7) ['workclass','education','marital-status','occupation','relationship','race','education-num_na']

In [ ]:
learn1.dls.classes

{'workclass': ['#na#', ' ?', ' Federal-gov', ' Local-gov', ' Never-worked', ' Private', ' Self-emp-inc', ' Self-emp-not-inc', ' State-gov', ' Without-pay'],
 'education': ['#na#', ' 10th', ' 11th', ' 12th', ' 1st-4th', ' 5th-6th', ' 7th-8th', ' 9th', ' Assoc-acdm', ' Assoc-voc', ' Bachelors', ' Doctorate', ' HS-grad', ' Masters', ' Preschool', ' Prof-school', ' Some-college'],
 'marital-status': ['#na#', ' Divorced', ' Married-AF-spouse', ' Married-civ-spouse', ' Married-spouse-absent', ' Never-married', ' Separated', ' Widowed'],
 'occupation': ['#na#', ' ?', ' Adm-clerical', ' Armed-Forces', ' Craft-repair', ' Exec-managerial', ' Farming-fishing', ' Handlers-cleaners', ' Machine-op-inspct', ' Other-service', ' Priv-house-serv', ' Prof-specialty', ' Protective-serv', ' Sales', ' Tech-support', ' Transport-moving'],
 'relationship': ['#na#', ' Husband', ' Not-in-family', ' Other-relative', ' Own-child', ' Unmarried', ' Wife'],
 'race': ['#na#', ' Amer-Indian-Eskimo', ' Asian-Pac-Island

In [ ]:
learn1.fit(1)

epoch     train_loss  valid_loss  accuracy  time    
0         0.375889    0.347853    0.838759  00:04     


In [ ]:
temp = 1

In [ ]:
#export
class TabTransfer:
    def __init__(self, old_learner, new_learner):
        self.old_cat_names = old_learner.dls.cat_names
        self.old_all_classes = old_learner.dls.classes
        self.old_learner = old_learner
        
        self.new_cat_names = new_learner.dls.cat_names
        self.new_all_classes = new_learner.dls.classes
        self.new_learner = new_learner
        
        
    def transfer(self, cat_names_to_transfer, verbose=False):
        self.transfer_list = cat_names_to_transfer
        for curr_cat in self.transfer_list:
            if not (curr_cat in self.old_cat_names and curr_cat in self.new_cat_names):
                continue
            old_cat_idx = self.old_cat_names.index(curr_cat)
            new_cat_idx = self.new_cat_names.index(curr_cat)
            
            
            
            # TODO: Make it so that this isn't required by taking care of this.
            try: assert (self.old_learner.model.embeds[old_cat_idx].embedding_dim == self.new_learner.model.embeds[new_cat_idx].embedding_dim)
            except: 
                print(f"Encountered an error for variable {curr_cat}: Make sure embeddings dimensions are same for {self.old_learner.model.embeds[old_cat_idx]} and {self.new_learner.model.embeds[new_cat_idx]}")
                print("Moving on to other cat vars")
                continue
            
            old_curr_classes = self.old_all_classes[curr_cat]
            new_curr_classes = self.new_all_classes[curr_cat]
            global temp
            temp = new_curr_classes
#             if old_curr_classes<new_curr_classes:
#                 sitch = 1
#             else if old_curr_classes=new_curr_classes:
#                 sitch = 2
#             else if old_curr_classes<new_curr_classes:
#                 sitch = 3
            
            weights_mean = self.old_learner.model.embeds[old_cat_idx].weight.mean(0)
            if verbose: print(f'mean is {weights_mean} for {self.old_learner.model.embeds[old_cat_idx].weight}')
                
#             switch(sitch):
#                 case 1: 

            # Case where some category in old, but not in new isn't being handled rn.

            for new_curr_class in new_curr_classes:
                new_curr_class_idx = new_curr_classes.o2i[new_curr_class]
                if verbose: print(f"{new_curr_class_idx}, {type(new_curr_class_idx)}")
                
                if new_curr_class in old_curr_classes:
                    old_curr_class_idx = old_curr_classes.o2i[new_curr_class]
                    if verbose: print(f'Transferring weights for class {new_curr_class}, cat {curr_cat} from previous weights')
                    if verbose: print(f"old weight for class is {self.new_learner.model.embeds[new_cat_idx].weight[new_curr_class_idx, :]}")
                    tempwgt1 = self.new_learner.model.embeds[new_cat_idx].weight[new_curr_class_idx, :]
                    tempwgt2 = self.old_learner.model.embeds[old_cat_idx].weight[old_curr_class_idx, :]
                    self.new_learner.model.embeds[new_cat_idx].weight.data[new_curr_class_idx, :] = self.old_learner.model.embeds[old_cat_idx].weight[old_curr_class_idx, :].detach().clone()
                    self.new_learner.model.embeds[new_cat_idx].weight[new_curr_class_idx, :].required_grad = True
                    if verbose: print(f"new weight for class is {self.new_learner.model.embeds[new_cat_idx].weight[new_curr_class_idx, :]}")
                else:
                    if verbose: print(f'Transferring weights for class {new_curr_class}, cat {curr_cat} using mean')
                    if verbose: print(f"old weight for class is {self.new_learner.model.embeds[new_cat_idx].weight[new_curr_class_idx, :]}")
                    self.new_learner.model.embeds[new_cat_idx].weight.data[new_curr_class_idx, :] = weights_mean
                    self.new_learner.model.embeds[new_cat_idx].weight[new_curr_class_idx, :].required_grad = True
                    if verbose: print(f"new weight for class is {self.new_learner.model.embeds[new_cat_idx].weight[new_curr_class_idx, :]}")

In [ ]:
learn1.model.embeds[0].embedding_dim

6

In [ ]:
tabobj = TabTransfer(learn1, learn2)

In [ ]:
tabobj.transfer(["race", "workclass"], verbose = True)

Encountered an error for variable race: Make sure embeddings dimensions are same for Embedding(6, 4) and Embedding(8, 5)
Moving on to other cat vars
mean is tensor([-0.0117,  0.0015, -0.0070, -0.0041, -0.0016, -0.0030],
       grad_fn=<MeanBackward1>) for Parameter containing:
tensor([[-0.0095, -0.0080, -0.0010,  0.0050,  0.0054,  0.0018],
        [-0.0257,  0.0145, -0.0246, -0.0116, -0.0239, -0.0175],
        [ 0.0356, -0.0378,  0.0255,  0.0302,  0.0010,  0.0535],
        [-0.0335, -0.0008, -0.0040, -0.0088,  0.0437, -0.0234],
        [-0.0035,  0.0144, -0.0053, -0.0161,  0.0081,  0.0213],
        [ 0.0246, -0.0193, -0.0273,  0.0203, -0.0426,  0.0175],
        [ 0.0327, -0.0590,  0.0567,  0.0367,  0.0288,  0.0402],
        [-0.0485,  0.0436, -0.0329, -0.0606, -0.0063, -0.0495],
        [-0.0470,  0.0217, -0.0159,  0.0074,  0.0009, -0.0349],
        [-0.0424,  0.0456, -0.0409, -0.0431, -0.0314, -0.0390]],
       requires_grad=True)
0, <class 'int'>
Transferring weights for class #na#, 